In [ ]:
from robomaster import robot
import time
import csv
import os
import math

# เก็บข้อมูล log
log_data_pos = []

# พารามิเตอร์ของ PD controller
Kp = 5.0                  # ค่า Proportional gain
Kd = 2.0                  # ค่า Derivative gain
tolerance = 0.01
current_x = 0.0
current_y = 0.0
prev_error = 0.0
prev_time = None

# เขียน CSV
import csv
import os

def write_csv_once(file_path, fieldnames, entry):
    """
    เขียนข้อมูล 1 แถวลงในไฟล์ CSV
    - สร้างไฟล์ใหม่ถ้ายังไม่มี
    - เขียน header แค่ครั้งแรก
    - เพิ่ม entry ทีละบรรทัด

    Parameters:
    - file_path (str): ที่อยู่ของไฟล์ CSV
    - fieldnames (list): รายชื่อ column ทั้งหมด
    - entry (dict): ข้อมูลที่จะเขียนลง 1 แถว
    """
    file_exists = os.path.isfile(file_path)
    
    with open(file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # เขียนหัวตารางถ้าไฟล์ใหม่หรือยังว่าง
        if not file_exists or os.path.getsize(file_path) == 0:
            writer.writeheader()
        
        # เขียนข้อมูล 1 แถว
        writer.writerow(entry)

# เก็บตำแหน่ง
def pos_info_handler(position_info):
    global current_x, current_y
    current_x = position_info[0]
    current_y = position_info[1]

def pd_move_to(ep_chassis, axis, target_value):
    global prev_error, prev_time
    prev_error = 0.0
    prev_time = None

    print(f"Moving on axis: {axis} to {target_value:.2f}m")
    while True:
        current_time = time.time()
        dt = 0.1 if prev_time is None else current_time - prev_time

        # เลือกตำแหน่งตามแกน
        pos = current_x if axis == "x" else current_y
        error = target_value - pos
        derivative = (error - prev_error) / dt if dt > 0 else 0.0

        speed = Kp * error + Kd * derivative
        speed = max(min(speed, 0.5), -0.5)  # จำกัดความเร็ว

        if abs(error) <= tolerance:
            ep_chassis.drive_speed(x=0, y=0, z=0)
            print(f"{axis}-axis target reached.")
            break

        # ควบคุมเฉพาะแกน
        if axis == "x":
            ep_chassis.drive_speed(x=speed, y=0, z=0)
        else:
            ep_chassis.drive_speed(x=0, y=speed, z=0)

        prev_error = error
        prev_time = current_time
        time.sleep(0.05)

def rotate(ep_chassis, angle_deg, speed_deg_per_sec=45):
    print(f"Rotating {angle_deg} degrees...")
    direction = 1 if angle_deg > 0 else -1
    ep_chassis.drive_speed(x=0, y=0, z=direction * speed_deg_per_sec)
    time.sleep(abs(angle_deg) / speed_deg_per_sec)
    ep_chassis.drive_speed(x=0, y=0, z=0)
    time.sleep(1)

if __name__ == '__main__':
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    ep_chassis = ep_robot.chassis
    ep_chassis.sub_position(freq=10, callback=pos_info_handler)

    try:
        # จุดเป้าหมายที่ต้องเคลื่อนไป
        square_path = [
            ("x", 1.2),
            ("y", 1.2),
            ("x", 0.0),
            ("y", 0.0)
        ]

        for axis, target in square_path:
            pd_move_to(ep_chassis, axis, target)
            rotate(ep_chassis, 90)

        print("✅ Square path complete.")

    finally:
        ep_chassis.unsub_position()
        ep_robot.close()
        print("Robot closed.")


: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# อ่านไฟล์ CSV ทั้งสอง
df_high = pd.read_csv("log_poshigh.csv")
df_low = pd.read_csv("log_poslow.csv")

# ดึงข้อมูล timestamp และตำแหน่ง x
time_high = df_high['timestamp']
x_high = df_high['x']

time_low = df_low['timestamp']
x_low = df_low['x']

# สร้างกราฟ subplot แบบแนวตั้ง (2 แถว 1 คอลัมน์)
plt.figure(figsize=(10, 12))

# กราฟ High (แกน X = Time, แกน Y = Position)
plt.subplot(2, 1, 1)
plt.plot(time_high, x_high, color='red')
plt.xlabel("Time (s)")
plt.ylabel("Position X (m)")
plt.title("Position X over Time - High")
plt.grid(True)

# กราฟ Low (แกน X = Time, แกน Y = Position)
plt.subplot(2, 1, 2)
plt.plot(time_low, x_low, color='blue')
plt.xlabel("Time (s)")
plt.ylabel("Position X (m)")
plt.title("Position X over Time - Low")
plt.grid(True)

# จัด layout ให้ดูดี
plt.tight_layout()

# แสดงผล
plt.show()
